# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is a classification problem because we are predicting a categorical variable. Specifically for this case, we are predicting a binary variable: 1 for "yes" (representing a output that a student would likely pass the exam and does not need early intervention), and 0 for "no" (representing the other output that a student would likely fail the exam and does need early intervention).

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [176]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [177]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = len(student_data[student_data["passed"] == "yes"])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == "no"])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/(float(n_failed) + float(n_passed))

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(100*grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [178]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [179]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [202]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test,
                                                    random_state = 42)


# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])
print "Grad rate of the training set: {:.2f}%".format(100 * (y_train == 'yes').mean())
print "Grad rate of the testing set: {:.2f}%".format(100 * (y_test == 'yes').mean())

Training set has 300 samples.
Testing set has 95 samples.
Grad rate of the training set: 68.33%
Grad rate of the testing set: 63.16%


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **
Since we are dealing with a classification problem, I choose three models: SVM, logistic regression and AdaBoost:


#####################  Support Vector Machine (SVM) 

GENERAL APPLICATIONS

SVM are useful in text and hypertext categorization, classification of images, handwritten characters recognition, and biology 
https://en.wikipedia.org/wiki/Support_vector_machine#Applications

STRENGTHS

SVM works well in a data set that there is clear margin of operation

WEAKNESSES

SVM doesn’t work well in large data set because of long training time. Also SVM model doesn’t work well with a lot noises. In addition, it is hard to interpret a SVM model

REASONS FOR CHOOSING

The  dataset has only 395 observations while the number of features is 30, so the dimension is high. SVM works well with high-dimensional data and avoids the curse of dimensionality problem ( Tan, Steinbach, and Kumar, 2006) 


#####################  Logistic Regression Model

GENERAL APPLICATIONS

Logistic regression is widely used in marketing analytics to predict probability whether a customer is likely to renew subscribes, or purchase new products based on CRM data ( his/her historical purchasing data and demographic data).  Logistic regression model is also widely used in financial industry, for example, whether to approve or reject an applicant’s credit card application based his/her credit history and other informations.

STRENGTHS

Simple to implement and easy to interpret. The training time is much less because of simplicity. It is also less prone to over-fitting

WEAKNESSES

Can’t handle the dataset that has a few highly corrected predictor variables

REASONS FOR CHOOSING

Easy to interpret the model and predicted results. Easy to implement without requiring a lot of computing resource


#####################  AdaBoost

GENERAL APPLICATIONS

AdaBoost is used in pattern recognition, specifically in facial recognition

STRENGTHS

AdaBoost is very simple to implement and runs faster than SVM because it uses weak learner. The performance is much better than the week learners because the final classifications are based on weighted vote of weak learners


WEAKNESSES

It is also sensitive to noises and outliers.S ince AdaBoosting has tendency to focus on training maples that are incorrectly classified, it can be prone to overfitting

REASONS FOR CHOOSING

Requires less computing resource than SVM with great performance

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [188]:
from time import time

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [182]:
# create a new function that produce a table showing training time, prediction time, 
# F1 score on training set and F1 score on test set, for each training set size.
import time 

from IPython.display import display, HTML


def train_test_models(clf, feature_train, feature_test, target_train, traget_test):
    
    Training_size = []
    Training_time = []
    Predict_time_training = []
    Predict_time_test = []
    F1_score_training = []
    F1_score_test = []


    for n in range(100, 400, 100):
        # training set size
        Training_size.append(n)

        # train the model and calculate the training time
        start_training = time.time()
        clf = clf.fit(X_train[0:n], y_train[0:n])
        end_training = time.time()
        training_time = end_training - start_training
        # training time
        Training_time.append(training_time)

        # predict using the training set
        start_pred_training = time.time()
        y_pred_train = clf.predict(X_train[0:n])
        end_pred_training = time.time()
        predict_time_training = end_pred_training - start_pred_training
        # prediction time in the training set
        Predict_time_training.append(predict_time_training)

        # predict using the test set
        start__pred_test = time.time()
        y_pred_test = clf.predict(X_test)
        end__pred_test = time.time()
        predict_time_test = end__pred_test - start__pred_test
        Predict_time_test.append(predict_time_test)


        # F1 score for the training and test set
        F1_score_training.append(f1_score(y_train[0:n].values, y_pred_train, pos_label='yes'))
        F1_score_test.append(f1_score(y_test.values, y_pred_test, pos_label= 'yes'))


    # make a dataframe and contains the output values
    data = {"Training Set Size": Training_size,
            "Training Time": Training_time,
            "Prediction Time(train)": Predict_time_training,
            "Prediction Time(test)": Predict_time_test,
            "F1 Score(train)": F1_score_training,
            "F1 Score(test)": F1_score_test}

    df = pd.DataFrame(data, columns =['Training Set Size', 'Training Time', "Prediction Time(train)", 'Prediction Time(test)', 'F1 Score(train)', 'F1 Score(test)'])

    display(HTML(df.to_html(index=False)))

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

In [183]:
# Train a SVM model using default parameters
from sklearn.svm import SVC
clf_svm = SVC()
train_test_models(clf_svm, X_train, X_test, y_train, y_test)

Training Set Size,Training Time,Prediction Time(train),Prediction Time(test),F1 Score(train),F1 Score(test)
100,0.002358,0.001314,0.000754,0.877698,0.774648
200,0.003341,0.002191,0.001475,0.867925,0.781457
300,0.007604,0.006295,0.002001,0.876068,0.783784


In [184]:
# Train a logistic regression model using default parameters
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression()
train_test_models(clf_lr, X_train, X_test, y_train, y_test)

Training Set Size,Training Time,Prediction Time(train),Prediction Time(test),F1 Score(train),F1 Score(test)
100,0.002028,0.000705,0.000522,0.859259,0.764706
200,0.002112,0.000442,0.000113,0.856187,0.791367
300,0.002747,0.000666,0.000192,0.846847,0.805970


In [186]:
# Train a AdaBoost model using decision tree as the weak learner, and default parameters
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()
clf_ada = AdaBoostClassifier(base_estimator = tree)
train_test_models(clf_ada, X_train, X_test, y_train, y_test)

Training Set Size,Training Time,Prediction Time(train),Prediction Time(test),F1 Score(train),F1 Score(test)
100,0.003139,0.000702,0.000266,1,0.626087
200,0.002519,0.000575,0.000254,1,0.736000
300,0.003397,0.000593,0.000251,1,0.650407


## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** Based on the outputs above, I would choose logistic regression as the best model for the student intervention system. 

Logistic regression gives the shortest prediction time while has the relatively highest F1 score in test data sets. It requires the least computing resource and has the best generalization. We will see the best performance of prediction and response time when deploy the model to as a web based system. SVM has the highest F1 score in training dataset, but require much more computing resource and has worse F1 score in test set than the logistic regression model. It is also hard to interpret SVM to non-technical decision makers. AdaBoosting has severe overfitting issue becasue of the perfect F1 score in training set and 0.65 F1 score in test set.

In addition, logistic regression model allows us to explain relationships between the binary response variable and predictor variables. So it will help us to find out how the predict variables impact the probability of whether a student would likely pass or fail. SVM and AdaBoosting, on the other hand, are more like black box and do not provide provide these informations. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

Logistic regression predicts whether a student would pass or fail the exam based on probability or likelihood.

It takes features about past student and learn to assign weight to these features to train a linear model.  With the linear model, we can plug in the features of student by multiplying each weights with the corresponding features value. The sum of the feature values multiplied by the weight in the equation gives us the log-odd value. With a sample conversion, we can estimate the probability whether or not the student would pass the exam. If the probability is greater than 0.5, we will predict that the student would likely pass the exam.  If the probability is less than 0.5, we will predict that the  student would likely fail the exam. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [189]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report

# TODO: Create the parameters list you wish to tune
parameters = [{'C': [0.01, 0.1, 1, 10, 100]}]

# TODO: Initialize the classifier
clf = LogisticRegression()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label = "yes", greater_is_better = True )

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=LogisticRegression(), param_grid= parameters, scoring=f1_scorer, cv = 5)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print clf
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

# look at the model's perfromance with a classification report
print classification_report(y_test, clf.predict(X_test))

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)
Made predictions in 0.0004 seconds.
Tuned model has a training F1 score of 0.8249.
Made predictions in 0.0002 seconds.
Tuned model has a testing F1 score of 0.7867.
             precision    recall  f1-score   support

         no       0.80      0.11      0.20        35
        yes       0.66      0.98      0.79        60

avg / total       0.71      0.66      0.57        95



### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

Answer:  The fine tuned logistic regression model uses 0.01 C value and it gives F1 score of 0.8249 in the training set and 0.7857 in the test set.  

Although the untuned logistic regression model with C = 1 gives slightly better F1 score in the training and test datasets, we believe that the fine tuned logistic model using 0.01 C value should be the final model because the GridSearch function uses 5-fold cross validation while the untuned model only use one train_test split. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.